# big_dix

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
def money(df,money,modem='Home'):
    for i in range(len(df)):
        bet_money=0.1*money
        money-=bet_money
        if df.iloc[i]['Won/NotWon']==1:
            if modem=='Home':
                add = bet_money * df.iloc[i]['ODDH_Aver.']
                money+=add
            elif modem=='Away':
                add = bet_money * df.iloc[i]['ODDA_Aver.']
                money+=add
            elif modem=='Draw':
                add = bet_money * df.iloc[i]['ODDD_Aver.']
                money+=add
            else:
                raise('aaaa')
    return money


def get_return_money(repos,top=5,modem='Home'):
    moneyz=100
    DF = go_get(repos,mode=2,lista=['OverAll'])
    method = list(DF['Method'])
    profit=[]
    for i in range(len(method)):
        tree,foret = method[i].split('_')
        listaz = ['Tree_Forest',tree,foret,'Predicted']
        df = go_get(repos,mode=2,lista=listaz)
        cash = round((100*(money(df,moneyz,modem=modem)/moneyz))-100,2)
        profit+=[[tree+'_'+foret,cash]]
    topz=[]
    for t in range(top):
        only_cash = np.array([profit[i][1] for i in range(len(profit))])
        ind_max=np.argmax(only_cash)
        topz+=[[profit[ind_max][0],profit[ind_max][1]]]
        del profit[ind_max]
    return topz

In [1]:
def weekly_profit(repos,w=4): #w é o número de semanas 
    old_df = go_get(repos,mode=2,lista=['Tree_Forest','TL50','FL50','ALL']).sort_values(['Date'],ascending=['True']).reset_index(drop=True)
    date0 = old_df.iloc[0]['Date']
    date=date0
    date_ = old_df.iloc[-1]['Date']
    n = 1
    BIG = pd.DataFrame({})
    DF = go_get(repos,mode=2,lista=['OverAll'])
    method = list(DF['Method'])
    method_cash = [100]*len(method)
    count=0
    end=False
    while not(end):
        if count==1:
            pre_date = date
            pos_date = date_
            end = True
        else:
            pre_date = date + (dt.timedelta(weeks=(w*(n-1))))
            pos_date = date + (dt.timedelta(weeks=(w*n)))
        if pos_date>=date_:
            count=1
        ok2 = pd.DataFrame({})
        for i in range(len(method)):
            tree,foret = method[i].split('_')
            listaz = ['Tree_Forest',tree,foret,'Predicted']
            df = go_get(repos,mode=2,lista=listaz)
            if count==0:
                df = df.loc[np.logical_and(df['Date']<pos_date,df['Date']>=pre_date)]
            elif (count==1) or (end):
                df = df.loc[np.logical_and(df['Date']<=pos_date,df['Date']>=pre_date)]
            cash = round(money(df,method_cash[i],modem='Home'),2)
            if n==1 and not(end):
                ok1 = pd.DataFrame({'Method':[tree+'_'+foret],'Week '+str(w*n):[round(cash/method_cash[i],2)]})
            elif n>1 and not(end):
                ok1 = pd.DataFrame({'Week '+str(w*n):[round(cash/method_cash[i],2)]})
            else: 
                ok1 = pd.DataFrame({'Total':[round(cash/method_cash[i],2)]})
            ok2 = pd.concat([ok2,ok1],sort=False,axis=0)
            method_cash[i]=cash
        BIG = pd.concat([BIG,ok2],sort=False,axis=1)
        n+=1
    return BIG.reset_index(drop=True)
        

In [ ]:
def model_std(repos): #devolve a maior sequência d 0s e 1s, e a std de 0s e 1s
    STD = pd.DataFrame({})
    DF = go_get(repos,mode=2,lista=['OverAll'])
    method = list(DF['Method'])
    for i in range(len(method)):
        tree,foret = method[i].split('_')
        listaz = ['Tree_Forest',tree,foret,'Predicted']
        df = go_get(repos,mode=2,lista=listaz)
        wnw = list(df['Won/NotWon'])
        lista_1 = [0]
        lista_0 = [0]
        current = 0
        for n in range(len(wnw)):
            if n==0:
                if wnw[n]==0:
                    lista_0[-1]+=1
                    current=0
                else:
                    lista_1[-1]+=1
                    current=1
            else:
                if current==0:
                    if wnw[n]==0:
                        lista_0[-1]+=1
                    else:
                        current=1
                        lista_1+=[1]
                elif current==1:
                    if wnw[n]==0:
                        current=0
                        lista_0+=[1]
                    else:
                        lista_1[-1]+=1
        Max0 = max(lista_0)
        Max1 = max(lista_1)
        aver0 = np.mean(lista_0)
        aver1 = np.mean(lista_1)
        std0 = round(sum(list(map(lambda x:abs(x),list(map(lambda x:x-aver0,lista_0))))))
        std1 = round(sum(list(map(lambda x:abs(x),list(map(lambda x:x-aver1,lista_1))))))
        ok1 = pd.DataFrame({'Method':[tree+'_'+foret],'Max0':[Max0],'Max1':[Max1],
                            'Aver_0':[round(aver0,1)],'Aver_1':[round(aver1,1)],
                            #'Count_0':[len(lista_0)],'Count_1':[len(lista_1)],
                           'STD_0':[std0],'STD_1':[std1]})
        STD = pd.concat([STD,ok1],axis=0,sort=False)
    return STD

In [ ]:
def Analyse_Model(repos,old_df,w=4):
    df1 = weekly_profit(repos,w=w)
    df2 = model_std(repos_old_df)
    DF = pd.concat([df1,df2],axis=1,sort=False)
    return DF

In [6]:
a = pd.DataFrame({'a':[12,34]})
b = pd.DataFrame({'a':[1,4]})

In [10]:
pd.concat([a,b],axis=0,sort=False).reset_index(drop=True)

,a
0,12
1,34
2,1
3,4
